# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 2:20PM,239153,16,ACG-2102490272,ACG North America LLC,Released
1,Jun 8 2022 2:20PM,239152,10,0085749144,ISDIN Corporation,Released
2,Jun 8 2022 2:20PM,239152,10,0085745619,ISDIN Corporation,Released
3,Jun 8 2022 2:20PM,239152,10,0085745721,ISDIN Corporation,Released
4,Jun 8 2022 2:14PM,239151,19,ADV80003167,"AdvaGen Pharma, Ltd",Released
5,Jun 8 2022 2:14PM,239151,19,ADV80003168,"AdvaGen Pharma, Ltd",Released
6,Jun 8 2022 2:14PM,239151,19,ADV80003169,"AdvaGen Pharma, Ltd",Released
7,Jun 8 2022 2:14PM,239151,19,ADV80003170,"AdvaGen Pharma, Ltd",Released
8,Jun 8 2022 2:14PM,239151,19,ADV80003171,"AdvaGen Pharma, Ltd",Released
9,Jun 8 2022 2:14PM,239151,19,ADV80003172,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,239149,Released,1
46,239150,Released,1
47,239151,Released,45
48,239152,Released,3
49,239153,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239149,NaN,NaN,1.0
239150,NaN,NaN,1.0
239151,NaN,NaN,45.0
239152,NaN,NaN,3.0
239153,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239019,29.0,5.0,0.0
239039,0.0,0.0,2.0
239045,22.0,7.0,3.0
239047,0.0,0.0,1.0
239050,3.0,4.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239019,29,5,0
239039,0,0,2
239045,22,7,3
239047,0,0,1
239050,3,4,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239019,29,5,0
1,239039,0,0,2
2,239045,22,7,3
3,239047,0,0,1
4,239050,3,4,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239019,29,5,
1,239039,,,2
2,239045,22,7,3
3,239047,,,1
4,239050,3,4,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 2:20PM,239153,16,ACG North America LLC
1,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation
4,Jun 8 2022 2:14PM,239151,19,"AdvaGen Pharma, Ltd"
49,Jun 8 2022 2:09PM,239150,21,"NBTY Global, Inc."
50,Jun 8 2022 1:51PM,239149,22,"NBTY Global, Inc."
51,Jun 8 2022 1:49PM,239148,22,"NBTY Global, Inc."
52,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.
58,Jun 8 2022 1:27PM,239144,19,ACG North America LLC
59,Jun 8 2022 1:24PM,239143,16,American International Chemical
60,Jun 8 2022 1:03PM,239140,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 2:20PM,239153,16,ACG North America LLC,,,1
1,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation,,,3
2,Jun 8 2022 2:14PM,239151,19,"AdvaGen Pharma, Ltd",,,45
3,Jun 8 2022 2:09PM,239150,21,"NBTY Global, Inc.",,,1
4,Jun 8 2022 1:51PM,239149,22,"NBTY Global, Inc.",,,1
5,Jun 8 2022 1:49PM,239148,22,"NBTY Global, Inc.",,,1
6,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.,,,6
7,Jun 8 2022 1:27PM,239144,19,ACG North America LLC,,,1
8,Jun 8 2022 1:24PM,239143,16,American International Chemical,,,1
9,Jun 8 2022 1:03PM,239140,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:20PM,239153,16,ACG North America LLC,1,,
1,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation,3,,
2,Jun 8 2022 2:14PM,239151,19,"AdvaGen Pharma, Ltd",45,,
3,Jun 8 2022 2:09PM,239150,21,"NBTY Global, Inc.",1,,
4,Jun 8 2022 1:51PM,239149,22,"NBTY Global, Inc.",1,,
5,Jun 8 2022 1:49PM,239148,22,"NBTY Global, Inc.",1,,
6,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.,6,,
7,Jun 8 2022 1:27PM,239144,19,ACG North America LLC,1,,
8,Jun 8 2022 1:24PM,239143,16,American International Chemical,1,,
9,Jun 8 2022 1:03PM,239140,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:20PM,239153,16,ACG North America LLC,1,,
1,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation,3,,
2,Jun 8 2022 2:14PM,239151,19,"AdvaGen Pharma, Ltd",45,,
3,Jun 8 2022 2:09PM,239150,21,"NBTY Global, Inc.",1,,
4,Jun 8 2022 1:51PM,239149,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:20PM,239153,16,ACG North America LLC,1.0,NaN,NaN
1,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation,3.0,NaN,NaN
2,Jun 8 2022 2:14PM,239151,19,"AdvaGen Pharma, Ltd",45.0,NaN,NaN
3,Jun 8 2022 2:09PM,239150,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jun 8 2022 1:51PM,239149,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 2:20PM,239153,16,ACG North America LLC,1.0,0.0,0.0
1,Jun 8 2022 2:20PM,239152,10,ISDIN Corporation,3.0,0.0,0.0
2,Jun 8 2022 2:14PM,239151,19,"AdvaGen Pharma, Ltd",45.0,0.0,0.0
3,Jun 8 2022 2:09PM,239150,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jun 8 2022 1:51PM,239149,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2630158,48.0,24.0,37.0
102,478176,2.0,0.0,0.0
12,239083,1.0,0.0,0.0
15,717142,9.0,11.0,25.0
16,717406,2.0,1.0,0.0
18,239107,1.0,0.0,0.0
19,1195484,50.0,23.0,11.0
20,956478,41.0,12.0,0.0
21,478272,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2630158,48.0,24.0,37.0
1,102,478176,2.0,0.0,0.0
2,12,239083,1.0,0.0,0.0
3,15,717142,9.0,11.0,25.0
4,16,717406,2.0,1.0,0.0
5,18,239107,1.0,0.0,0.0
6,19,1195484,50.0,23.0,11.0
7,20,956478,41.0,12.0,0.0
8,21,478272,2.0,0.0,0.0
9,22,478297,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,48.0,24.0,37.0
1,102,2.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,9.0,11.0,25.0
4,16,2.0,1.0,0.0
5,18,1.0,0.0,0.0
6,19,50.0,23.0,11.0
7,20,41.0,12.0,0.0
8,21,2.0,0.0,0.0
9,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,48.0
1,102,Released,2.0
2,12,Released,1.0
3,15,Released,9.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21,22
Status,,,,,,,,,,
Completed,37.0,0.0,0.0,25.0,0.0,0.0,11.0,0.0,0.0,0.0
Executing,24.0,0.0,0.0,11.0,1.0,0.0,23.0,12.0,0.0,0.0
Released,48.0,2.0,1.0,9.0,2.0,1.0,50.0,41.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,37.0,0.0,0.0,25.0,0.0,0.0,11.0,0.0,0.0,0.0
1,Executing,24.0,0.0,0.0,11.0,1.0,0.0,23.0,12.0,0.0,0.0
2,Released,48.0,2.0,1.0,9.0,2.0,1.0,50.0,41.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,37.0,0.0,0.0,25.0,0.0,0.0,11.0,0.0,0.0,0.0
1,Executing,24.0,0.0,0.0,11.0,1.0,0.0,23.0,12.0,0.0,0.0
2,Released,48.0,2.0,1.0,9.0,2.0,1.0,50.0,41.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()